In [2]:
!pip install -q git+https://github.com/openai/whisper.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.9 MB/s eta 0:00:00


In [3]:
!sudo apt update && sudo apt install -y ffmpeg


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,078 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,092 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadco

In [16]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.7 MB/s eta 0:00:00


In [20]:
import whisper
from moviepy.editor import VideoFileClip
from rapidfuzz import fuzz

def extract_audio(video_path, audio_path="audio.wav"):
    """
    Extract audio from video file.
    """
    try:
        print(" Extracting audio from video...")
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_path, verbose=False, logger=None)
        return audio_path
    except Exception as e:
        print("Error extracting audio:", e)
        return None

def transcribe_audio(audio_path, model_name="base"):
    """
    Transcribe audio using Whisper and return transcript segments.
    """
    print("Transcribing audio using Whisper...")
    model = whisper.load_model(model_name)
    result = model.transcribe(audio_path)
    return result['segments']

def print_transcript_segments(segments):
    """
    Print transcript segments for inspection.
    """
    print("\n--- Transcript Segments ---")
    for segment in segments:
        start = segment['start']
        end = segment['end']
        text = segment['text']
        print(f"[{start:.2f}s - {end:.2f}s]: {text}")

def get_timestamp_for_query(video_path, query, similarity_threshold=70, model_name="base"):
    """
    Extract audio, transcribe, print segments, and search for fuzzy query match.
    Returns start time in seconds if found, else None.
    """
    audio_path = extract_audio(video_path)
    if audio_path is None:
        return None

    segments = transcribe_audio(audio_path, model_name=model_name)
    print_transcript_segments(segments)

    query_lower = query.lower()

    for segment in segments:
        segment_text = segment['text'].lower()
        similarity = fuzz.partial_ratio(query_lower, segment_text)
        if similarity >= similarity_threshold:
            print(f"\nMatch found with similarity {similarity}%:")
            print(f"Segment text: {segment['text']}")
            return segment['start']

    print("\n Query not found in transcript.")
    return None

def format_timestamp(seconds):
    """
    Convert seconds to hh:mm:ss string.
    """
    if seconds is None:
        return "Not found"
    hrs = int(seconds // 3600)
    mins = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hrs:02d}:{mins:02d}:{secs:02d}"

if __name__ == "__main__":
    video_path = "/content/talk_video.mp4"
    query = "You said he wants to erode the very fabric of civilization and Soros hates humanity."

    timestamp = get_timestamp_for_query(video_path, query, similarity_threshold=60, model_name="base")

    print("\n Timestamp (seconds):", timestamp)
    print("Timestamp (hh:mm:ss):", format_timestamp(timestamp))


 Extracting audio from video...
Transcribing audio using Whisper...


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")




--- Transcript Segments ---
[0.00s - 5.92s]:  We got easy, we got hard, and we've got impossible. Level one easy, hard mode.
[11.84s - 12.88s]:  Level impossible.
[16.00s - 16.40s]:  Right!
[19.92s - 21.60s]:  I think it's time to bring in the big guns.
[21.60s - 24.40s]:  Zach, stand! Line change.
[24.40s - 26.96s]:  All right, let's go boys. Time for impossible modes.
[30.00s - 31.20s]:  Get me going.
[31.20s - 32.80s]:  Yes, we're doing it.
[32.80s - 34.80s]:  You get me going.
[34.80s - 36.80s]:  A superhuman.
[36.80s - 40.80s]:  And I want more, more, more of what you're doing.
[40.80s - 42.80s]:  You get me going.
[42.80s - 44.00s]:  Zach, let's go!

 Query not found in transcript.

⏱ Timestamp (seconds): None
⏰ Timestamp (hh:mm:ss): Not found
